1) Design model (input, output size,forward pass)\
2) Construct loss and optimizer\
3) Training loop\
   ----forward pass: compute prediction\
   ----backward pass: gradients\
   ----update weights... iterate this a couple of times\
4) We replace our forward method with a pytorch module, and we don't need our weights. pytorch knows our parameter

In [2]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [7]:
# 0) prepare the data
bc = datasets.load_breast_cancer()
X,y = bc.data, bc.target

n_samples, n_features = X.shape

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state = 42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

y_train = y_train.view(y_train.shape[0],1) #we want to have only 1 column vector
y_test = y_test.view(y_test.shape[0],1) #we want to have only 1 column vector

# 1) model
# f = wx + b, sigmoid at the end

class LogisticRegression(nn.Module):
    def __init__(self, n_input_features):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(n_input_features, 1)
    def forward(self, x):
        y_predicted = torch.sigmoid(self.linear(x))
        return y_predicted
    
model = LogisticRegression(n_features)

# 2) loss and optimizer
learning_rate = 0.01
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

# 3) training loop
num_epochs = 100
for epoch in range(num_epochs):
    #forward pass and loss
    y_predicted = model(X_train)
    loss = criterion(y_predicted, y_train)
    
    #backward pass
    loss.backward()
    
    #updates
    optimizer.step()
    #empty gradient
    optimizer.zero_grad() #backward function will always add up all the gradients into the the grad att.
    
    if (epoch+1) % 10 == 0:
        print(f'epoch: {epoch+1}, loss = {loss.item():.4f}')
with torch.no_grad(): #evaluation should not be part of computational graph where we track history
    y_predicted = model(X_test) #get all predicted class from our test sample
    y_predicted_class = y_predicted.round() #convert to class label, sigmoid returns value between 0 and 1
    acc = y_predicted_class.eq(y_test).sum()/float(y_test.shape[0])
    print(f'accuracy = {acc:.4f}')
    

epoch: 10, loss = 0.6659
epoch: 20, loss = 0.5377
epoch: 30, loss = 0.4590
epoch: 40, loss = 0.4063
epoch: 50, loss = 0.3684
epoch: 60, loss = 0.3398
epoch: 70, loss = 0.3173
epoch: 80, loss = 0.2990
epoch: 90, loss = 0.2839
epoch: 100, loss = 0.2710
accuracy = 0.9737
